In [1]:
%load_ext autoreload
%autoreload 2

from deep_bf.data_handler import DataLoader
import torch
import numpy as np
from tqdm import tqdm
import h5py
import hdf5plugin

data_path = "/home/panda/rf_data/dataset/raw"
img_path = "/home/panda/rf_data/dataset/img"
samples_idx_path = "/home/panda/rf_data/dataset/samples_idx"

dl = DataLoader("/home/panda/rf_data/")

nz = 2048
nx = 256
new_Ns = 2800


device = "cuda"
dtype = torch.float32

In [2]:
df = dl.get_df()
df = df.query("RF == 1 and n_channels == 128")
df = df[df["name"].str[:3] != "JHU"]

df2 = df.sort_values("name")
df2 = df2.query("RF == 1 and n_channels == 128 and source == 'CUBDL'")

keys = ["n_angles", "sampling_frecuency", "aperture_width", "element_width", "pitch", "image_depth", "n_channels", "zlims", "center_frecuency"]
group = df2.groupby(keys)


In [3]:
from deep_bf.wrapper import BenchmarkWrapper

beamformer = "MVB"
bfs = [beamformer]

id = 0
for group_name, mini_df in group:
    name = mini_df["name"].iloc[0]

    pw = dl.get_defined_pwdata(name, "RF")
    f_num = 1.75
    window = "tukey25"

    angles_idx = np.arange(pw.n_angles)

    bw = BenchmarkWrapper(nz, nx, pw, angles_idx, bfs, f_num, window, device=device, dtype=dtype)
    o = bw.compute_reconstructions()[bfs[0]]

    angle = pw.n_angles // 2
    samples_idx = bw.get_samples_idx_by_angle(angle)
    samples_idx = samples_idx.cpu()

    with h5py.File(f"{samples_idx_path}/{id}.hdf5", "w") as f:
        f.create_dataset("samples_idx", data=samples_idx, chunks=True, **hdf5plugin.Bitshuffle(nelems=0, cname="zstd"))

    id += 1

In [4]:
for group_name, mini_df in group:
    gn = mini_df["name"]
    print(gn.iloc[0], gn.iloc[-1], gn.count())

TSH002 TSH501 500
OSL010 OSL010 1
INS001 INS013 13
OSL002 OSL002 1
OSL003 OSL003 1
OSL004 OSL004 1
OSL005 OSL005 1
OSL006 OSL006 1
OSL007 OSL007 1
MYO001 MYO006 6
INS014 INS026 13
UFL001 UFL005 5
